# Notebook Initilization

In [1]:
# System Imports #

import os
import itertools
import json

import pandas as pd
import networkx as nx

from tqdm import tqdm
from pyvis.network import Network

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Local Imports #

os.chdir("..")  # change current directory to project source code folder

from llmgrapher.helpers import prompts

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
SYS_PROMPT = (
        "You are a knowledge graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]\n"
        "Do not add any other comment before or after the json. Respond ONLY with a well formed json that can be directly read by a program."
)

In [4]:
print(SYS_PROMPT)

You are a knowledge graph maker who extracts terms and their relations from a given context. You are provided with a context chunk (delimited by ```) Your task is to extract the ontology of terms mentioned in the given context. These terms should represent the key concepts as per the context. 
Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.
	Terms may include object, entity, location, organization, person, 
	condition, acronym, documents, service, concept, etc.
	Terms should be as atomistic as possible

Thought 2: Think about how these terms can have one on one relation with other terms.
	Terms that are mentioned in the same sentence or the same paragraph are typically related to each other.
	Terms can be related to many other terms

Thought 3: Find out the relation between each such related pair of terms. 

Format your output as a list of json. Each element of the list contains a pair of termsand the relation between them, like the follwin

In [5]:
text = """
___Text___

 = Du Fu = 
 
 Du Fu ( Wade – Giles : Tu Fu ; Chinese : 杜甫 ; 712 – 770 ) was a prominent Chinese poet of the Tang dynasty . Along with Li Bai ( Li Po ) , he is frequently called the greatest of the Chinese poets . His greatest ambition was to serve his country as a successful civil servant , but he proved unable to make the necessary accommodations . His life , like the whole country , was devastated by the An Lushan Rebellion of 755 , and his last 15 years were a time of almost constant unrest . 
 Although initially he was little @-@ known to other writers , his works came to be hugely influential in both Chinese and Japanese literary culture . Of his poetic writing , nearly fifteen hundred poems have been preserved over the ages . He has been called the " Poet @-@ Historian " and the " Poet @-@ Sage " by Chinese critics , while the range of his work has allowed him to be introduced to Western readers as " the Chinese Virgil , Horace , Ovid , Shakespeare , Milton , Burns , Wordsworth , Béranger , Hugo or Baudelaire " . 
 
 = = Life = = 
 
 Traditional Chinese literary criticism emphasized the life of the author when interpreting a work , a practice which Burton Watson attributes to " the close links that traditional Chinese thought posits between art and morality " . Since many of Du Fu 's poems feature morality and history , this practice is particularly important . Another reason , identified by the Chinese historian William Hung , is that Chinese poems are typically concise , omitting context that might be relevant , but which an informed contemporary could be assumed to know . For modern Western readers , " The less accurately we know the time , the place and the circumstances in the background , the more liable we are to imagine it incorrectly , and the result will be that we either misunderstand the poem or fail to understand it altogether " . Stephen Owen suggests a third factor particular to Du Fu , arguing that the variety of the poet 's work required consideration of his whole life , rather than the " reductive " categorizations used for more limited poets . 
 
 = = = Early years = = = 
 
 Most of what is known of Du Fu 's life comes from his poems . His paternal grandfather was Du Shenyan , a noted politician and poet during the reign of Empress Wu . Du Fu was born in 712 ; the exact birthplace is unknown , except that it was near Luoyang , Henan province ( Gong county is a favourite candidate ) . In later life , he considered himself to belong to the capital city of Chang 'an , ancestral hometown of the Du family . 
 Du Fu 's mother died shortly after he was born , and he was partially raised by his aunt . He had an elder brother , who died young . He also had three half brothers and one half sister , to whom he frequently refers in his poems , although he never mentions his stepmother . 
 The son of a minor scholar @-@ official , his youth was spent on the standard education of a future civil servant : study and memorisation of the Confucian classics of philosophy , history and poetry . He later claimed to have produced creditable poems by his early teens , but these have been lost . 
 In the early 730s , he travelled in the Jiangsu / Zhejiang area ; his earliest surviving poem , describing a poetry contest , is thought to date from the end of this period , around 735 . In that year , he took the civil service exam , likely in Chang 'an . He failed , to his surprise and that of centuries of later critics . Hung concludes that he probably failed because his prose style at the time was too dense and obscure , while Chou suggests his failure to cultivate connections in the capital may have been to blame . After this failure , he went back to traveling , this time around Shandong and Hebei . 
 His father died around 740 . Du Fu would have been allowed to enter the civil service because of his father 's rank , but he is thought to have given up the privilege in favour of one of his half brothers . He spent the next four years living in the Luoyang area , fulfilling his duties in domestic affairs . 
 In the autumn of 744 , he met Li Bai ( Li Po ) for the first time , and the two poets formed a friendship . David Young describes this as " the most significant formative element in Du Fu 's artistic development " because it gave him a living example of the reclusive poet @-@ scholar life to which he was attracted after his failure in the civil service exam . The relationship was somewhat one @-@ sided , however . Du Fu was by some years the younger , while Li Bai was already a poetic star . We have twelve poems to or about Li Bai from the younger poet , but only one in the other direction . They met again only once , in 745 . 
 In 746 , he moved to the capital in an attempt to resurrect his official career . He took the civil service exam a second time during the following year , but all the candidates were failed by the prime minister ( apparently in order to prevent the emergence of possible rivals ) . He never again attempted the examinations , instead petitioning the emperor directly in 751 , 754 and probably again in 755 . He married around 752 , and by 757 the couple had had five children — three sons and two daughters — but one of the sons died in infancy in 755 . From 754 he began to have lung problems ( probably asthma ) , the first of a series of ailments which dogged him for the rest of his life . It was in that year that Du Fu was forced to move his family due to the turmoil of a famine brought about by massive floods in the region . 
 In 755 , he received an appointment as Registrar of the Right Commandant 's office of the Crown Prince 's Palace . Although this was a minor post , in normal times it would have been at least the start of an official career . Even before he had begun work , however , the position was swept away by events . 
 
 = = = War = = = 
 
 The An Lushan Rebellion began in December 755 , and was not completely suppressed for almost eight years . It caused enormous disruption to Chinese society : the census of 754 recorded 52 @.@ 9 million people , but ten years later , the census counted just 16 @.@ 9 million , the remainder having been displaced or killed . During this time , Du Fu led a largely itinerant life unsettled by wars , associated famines and imperial displeasure . This period of unhappiness was the making of Du Fu as a poet : Even Shan Chou has written that , " What he saw around him — the lives of his family , neighbors , and strangers – what he heard , and what he hoped for or feared from the progress of various campaigns — these became the enduring themes of his poetry " . Even when he learned of the death of his youngest child , he turned to the suffering of others in his poetry instead of dwelling upon his own misfortunes . Du Fu wrote : 
 Brooding on what I have lived through , if even I know such suffering , the common man must surely be rattled by the winds . 
 In 756 , Emperor Xuanzong was forced to flee the capital and abdicate . Du Fu , who had been away from the city , took his family to a place of safety and attempted to join the court of the new emperor ( Suzong ) , but he was captured by the rebels and taken to Chang 'an . In the autumn , his youngest son , Du Zongwu ( Baby Bear ) , was born . Around this time Du Fu is thought to have contracted malaria . 
 He escaped from Chang 'an the following year , and was appointed Reminder when he rejoined the court in May 757 . This post gave access to the emperor but was largely ceremonial . Du Fu 's conscientiousness compelled him to try to make use of it : he caused trouble for himself by protesting the removal of his friend and patron Fang Guan on a petty charge . He was arrested but was pardoned in June . He was granted leave to visit his family in September , but he soon rejoined the court and on December 8 , 757 , he returned to Chang 'an with the emperor following its recapture by government forces . However , his advice continued to be unappreciated , and in the summer of 758 he was demoted to a post as Commissioner of Education in Huazhou . The position was not to his taste : in one poem , he wrote : 
 I am about to scream madly in the office / Especially when they bring more papers to pile higher on my desk . 
 He moved on in the summer of 759 ; this has traditionally been ascribed to famine , but Hung believes that frustration is a more likely reason . He next spent around six weeks in Qinzhou ( now Tianshui , Gansu province ) , where he wrote more than sixty poems . 
 
 = = = Chengdu = = = 
 
 In December 759 , he briefly stayed in Tonggu ( modern Gansu ) . He departed on December 24 for Chengdu ( Sichuan province ) , where he was hosted by local Prefect and fellow poet Pei Di . Du subsequently based himself in Sichuan for most of the next five years . By the autumn of that year he was in financial trouble , and sent poems begging help to various acquaintances . He was relieved by Yan Wu , a friend and former colleague who was appointed governor general at Chengdu . Despite his financial problems , this was one of the happiest and most peaceful periods of his life . Many of Du 's poems from this period are peaceful depictions of his life at " thatched hut " . In 762 , he left the city to escape a rebellion , but he returned in summer 764 when he was appointed an advisor to Yan , who was involved in campaigns against the Tibetan Empire . 
 
 = = = Last years = = = 
 
 Luoyang , the region of his birthplace , was recovered by government forces in the winter of 762 , and in the spring of 765 Du Fu and his family sailed down the Yangtze , apparently with the intention of making their way there . They traveled slowly , held up by his ill @-@ health ( by this time he was suffering from poor eyesight , deafness and general old age in addition to his previous ailments ) . They stayed in Kuizhou ( in what is now Baidicheng , Chongqing ) at the entrance to the Three Gorges for almost two years from late spring 766 . This period was Du Fu 's last great poetic flowering , and here he wrote 400 poems in his dense , late style . In autumn 766 , Bo Maolin became governor of the region : he supported Du Fu financially and employed him as his unofficial secretary . 
 In March 768 , he began his journey again and got as far as Hunan province , where he died in Tanzhou ( now Changsha ) in November or December 770 , in his 58th year . He was survived by his wife and two sons , who remained in the area for some years at least . His last known descendant is a grandson who requested a grave inscription for the poet from Yuan Zhen in 813 . 
 Hung summarises his life by concluding that , " He appeared to be a filial son , an affectionate father , a generous brother , a faithful husband , a loyal friend , a dutiful official , and a patriotic subject . " 
 Below is an example of one of Du Fu 's later works , To My Retired Friend Wei ( Chinese : 贈衛八處士 ) . Like many other poems in the Tang it featured the theme of a long parting between friends , which was often due to officials being frequently transferred to the provinces : 
 
 = = Works = = 
 
 Criticism of Du Fu 's works has focused on his strong sense of history , his moral engagement , and his technical excellence . 
 
 = = = History = = = 
 
 Since the Song dynasty , critics have called Du Fu the " poet historian " ( 詩史 shī shǐ ) . The most directly historical of his poems are those commenting on military tactics or the successes and failures of the government , or the poems of advice which he wrote to the emperor . Indirectly , he wrote about the effect of the times in which he lived on himself , and on the ordinary people of China . As Watson notes , this is information " of a kind seldom found in the officially compiled histories of the era " . 
 Du Fu 's political comments are based on emotion rather than calculation : his prescriptions have been paraphrased as , " Let us all be less selfish , let us all do what we are supposed to do " . Since his views were impossible to disagree with , his forcefully expressed truisms enabled his installation as the central figure of Chinese poetic history . 
 
 = = = Moral engagement = = = 
 
 A second favourite epithet of Chinese critics is that of " poet sage " ( 詩聖 shī shèng ) , a counterpart to the philosophical sage , Confucius . One of the earliest surviving works , The Song of the Wagons ( from around 750 ) , gives voice to the sufferings of a conscript soldier in the imperial army and a clear @-@ sighted consciousness of suffering . These concerns are continuously articulated in poems on the lives of both soldiers and civilians produced by Du Fu throughout his life . 
 Although Du Fu 's frequent references to his own difficulties can give the impression of an all @-@ consuming solipsism , Hawkes argues that his " famous compassion in fact includes himself , viewed quite objectively and almost as an afterthought " . He therefore " lends grandeur " to the wider picture by comparing it to " his own slightly comical triviality " . 
 Du Fu 's compassion , for himself and for others , was part of his general broadening of the scope of poetry : he devoted many works to topics which had previously been considered unsuitable for poetic treatment . Zhang Jie wrote that for Du Fu , " everything in this world is poetry " , Du wrote extensively on subjects such as domestic life , calligraphy , paintings , animals , and other poems . 
 
 = = = Technical excellence = = = 
 
 Du Fu 's work is notable above all for its range . Chinese critics traditionally used the term 集大成 ( jídàchéng- " complete symphony " ) , a reference to Mencius ' description of Confucius . Yuan Zhen was the first to note the breadth of Du Fu 's achievement , writing in 813 that his predecessor , " united in his work traits which previous men had displayed only singly " . He mastered all the forms of Chinese poetry : Chou says that in every form he " either made outstanding advances or contributed outstanding examples " . Furthermore , his poems use a wide range of registers , from the direct and colloquial to the allusive and self @-@ consciously literary . This variety is manifested even within individual works : Owen identifies the , " rapid stylistic and thematic shifts " in poems which enable the poet to represent different facets of a situation , while Chou uses the term " juxtaposition " as the major analytical tool in her work . Du Fu is noted for having written more on poetics and painting than any other writer of his time . He wrote eighteen poems on painting alone , more than any other Tang poet . Du Fu 's seemingly negative commentary on the prized horse paintings of Han Gan ignited a controversy that has persisted to the present day . 
 The tenor of his work changed as he developed his style and adapted to his surroundings ( " chameleon @-@ like " according to Watson ) : his earliest works are in a relatively derivative , courtly style , but he came into his own in the years of the rebellion . Owen comments on the " grim simplicity " of the Qinzhou poems , which mirrors the desert landscape ; the works from his Chengdu period are " light , often finely observed " ; while the poems from the late Kuizhou period have a " density and power of vision " . 
 Although he wrote in all poetic forms , Du Fu is best known for his lǜshi , a type of poem with strict constraints on form and content , for example : 
 About two thirds of Du Fu 's 1500 extant works are in this form , and he is generally considered to be its leading exponent . His best lǜshi use the parallelisms required by the form to add expressive content rather than as mere technical restrictions . Hawkes comments that , " it is amazing that Tu Fu is able to use so immensely stylized a form in so natural a manner " . 
 
 = = Influence = = 
 
 According to the Encyclopædia Britannica , Du Fu 's writings are considered by many literary critics to be among the greatest of all time , and it states " his dense , compressed language makes use of all the connotative overtones of a phrase and of all the intonational potentials of the individual word , qualities that no translation can ever reveal . " 
 In his lifetime and immediately following his death , Du Fu was not greatly appreciated . In part this can be attributed to his stylistic and formal innovations , some of which are still " considered extremely daring and bizarre by Chinese critics . " There are few contemporary references to him — only eleven poems from six writers — and these describe him in terms of affection , but not as a paragon of poetic or moral ideals . Du Fu is also poorly represented in contemporary anthologies of poetry . 
 However , as Hung notes , he " is the only Chinese poet whose influence grew with time " , and his works began to increase in popularity in the ninth century . Early positive comments came from Bai Juyi , who praised the moral sentiments of some of Du Fu 's works ( although he found these in only a small fraction of the poems ) , and from Han Yu , who wrote a piece defending Du Fu and Li Bai on aesthetic grounds from attacks made against them . Both these writers showed the influence of Du Fu in their own poetic work . By the beginning of the 10th century , Wei Zhuang constructed the first replica of his thatched cottage in Sichuan . 
 It was in the 11th century , during the Northern Song era that Du Fu 's reputation reached its peak . In this period a comprehensive re @-@ evaluation of earlier poets took place , in which Wang Wei , Li Bai and Du Fu came to be regarded as representing respectively the Buddhist , Daoist and Confucian strands of Chinese culture . At the same time , the development of Neo @-@ Confucianism ensured that Du Fu , as its poetic exemplar , occupied the paramount position . Su Shi famously expressed this reasoning when he wrote that Du Fu was " preeminent ... because ... through all his vicissitudes , he never for the space of a meal forgot his sovereign " . His influence was helped by his ability to reconcile apparent opposites : political conservatives were attracted by his loyalty to the established order , while political radicals embraced his concern for the poor . Literary conservatives could look to his technical mastery , while literary radicals were inspired by his innovations . Since the establishment of the People 's Republic of China , Du Fu 's loyalty to the state and concern for the poor have been interpreted as embryonic nationalism and socialism , and he has been praised for his use of simple , " people 's language " . 
 Du Fu 's popularity grew to such an extent that it is as hard to measure his influence as that of Shakespeare in England : it was hard for any Chinese poet not to be influenced by him . While there was never another Du Fu , individual poets followed in the traditions of specific aspects of his work : Bai Juyi 's concern for the poor , Lu You 's patriotism , and Mei Yaochen 's reflections on the quotidian are a few examples . More broadly , Du Fu 's work in transforming the lǜshi from mere word play into " a vehicle for serious poetic utterance " set the stage for every subsequent writer in the genre . 
 In the 20th century , he was the favourite poet of Kenneth Rexroth , who has described him as " the greatest non @-@ epic , non @-@ dramatic poet who has survived in any language " , and commented that , " he has made me a better man , as a moral agent and as a perceiving organism " . 
 
 = = = Influence on Japanese literature = = = 
 
 Du Fu 's poetry has made a profound impact on Japanese literature , especially on the literature from the Muromachi period and on scholars and poets in the Edo period , including Matsuo Bashō , the very greatest of all haiku poets . Even in modern Japanese , the term Saint of Poetry ( 詩聖 , shisei ) is mostly synonymous with Du Fu . 
 Until the 13th century , the Japanese preferred Bai Juyi above all poets and there were few references to Du Fu , although his influence can be seen in some kanshi ( " Chinese poetry made by Japanese poets " ) anthologies such as Bunka Shūreishū in the 9th century . The first notable Japanese appreciator of Du Fu 's poetry was Kokan Shiren ( 1278 – 1346 ) , a Rinzai Zen patriarch and one of the most prominent authors of the literature of the Five Mountains ; he highly praised Du Fu and made a commentary on some poems of Du Fu from the perspective of a Zen priest in Vol . 11 of Saihokushū . His student Chūgan Engetsu composed many kanshi which were clearly stated " influenced by Du Fu " in their prefaces . Chūgan 's student Gidō Shūshin had close connection with the Court and Ashikaga Shogunate and propagated Du Fu 's poetry in the mundane world ; one day Nijō Yoshimoto , the Kampaku regent of the Court and the highest authority of renga poetry , asked Gidō , " Should I learn the poetry of Du Fu and Li Bai ? " Gidō dared to reply , " Yes if you do have enough capability . No if do not . " Since then , there had been many seminars on Du Fu 's poetry both in Zen temples and in the aristocratic society , and as a result his poetry was often cited in Japanese literature in the Muromachi period , e.g. , Taiheiki , a historical epic in the late 14th century , and some noh plays such as Hyakuman , Bashō , and Shunkan . 
 During the Kan 'ei era of the Edo period ( 1624 – 1643 ) , Shào Chuán ( 邵傳 ) of the Ming Dynasty 's Collective Commentary on Du Fu 's Lǜshi ( 杜律集解 , Toritsu Shikkai ) was imported into Japan , and it gained explosive popularity in Confucian scholars and chōnin ( townspeople ) class . The commentary established Du Fu 's fame as the highest of all poets ; for instance , Hayashi Shunsai , a notable Confucian scholar , commented in Vol . 37 of Gahō Bunshū that Zǐměi [ Du Fu ] was the very best poet in history and praised Shào Chuán 's commentary for its simplicity and readability , while he criticized old commentaries during the Yuan Dynasty were too unfathomable . Matsuo Bashō , the greatest haiku poet , was also strongly influenced by Du Fu ; in Oku no Hosomichi , his masterpiece , he cites the first two lines of A Spring View ( 春望 ) before a haiku as its introduction and also many of his other haiku have similar wording and themes . It is said that when he died in Osaka during a long travel , a copy of Du Fu 's poetry was found with him as one of a few precious items which he was able to carry around . 
 
 = = Translation = = 
 
 A variety of styles have been used in efforts to translate Du Fu 's work into English . As Burton Watson remarks in The Selected Poems of Du Fu , " There are many different ways to approach the problems involved in translating Du Fu , which is why we need as many different translations as possible " ( p. xxii ) . The translators have had to contend with bringing out the formal constraints of the original without sounding laboured to a Western ear ( particularly when translating regulated verse , or lǜshi ) , and accommodating the complex allusions contained particularly in the later works ( Hawkes writes that " his poems do not as a rule come through very well in translation " — p. ix ) . One extreme on each issue is represented by Kenneth Rexroth 's One Hundred Poems From the Chinese . His are free translations , which seek to conceal the parallelisms through enjambement and expansion and contraction of the content ; his responses to the allusions are firstly to omit most of these poems from his selection , and secondly to " translate out " the references in those works which he does select . 
 Other translators have placed much greater weight on trying to convey a sense of the poetic forms used by Du Fu . Vikram Seth in Three Chinese Poets uses English @-@ style rhyme schemes , whereas Keith Holyoak in Facing the Moon approximates the Chinese rhyme scheme ; both use end @-@ stopped lines and preserve some degree of parallelism . In The Selected Poems of Du Fu , Burton Watson follows the parallelisms quite strictly , persuading the western reader to adapt to the poems rather than vice versa . Similarly , he deals with the allusion of the later works by combining literal translation with extensive annotation . 
 In 2015 , Stephen Owen published translations , with facing Chinese texts , of the complete poetry of Du Fu in six volumes , with extensive scholarly apparatus , which emphasized literalness . """

In [6]:
text.__len__()

24441

In [7]:
# Split the document to smaller chunks, because when giving a big chunk to the LLM, it may produce
# a summary of the given text rather than the requested graph as a JSON formatted text.
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

In [8]:
pages = splitter.split_text(text)

In [9]:
print("Number of Pages:", len(pages), "\n")
for i, page in enumerate(pages):
    print(f"###### Page {i} ######:\n{page}\n###### ------ ######\n\n")

Number of Pages: 21 

###### Page 0 ######:
___Text___
###### ------ ######


###### Page 1 ######:
= Du Fu = 
 
 Du Fu ( Wade – Giles : Tu Fu ; Chinese : 杜甫 ; 712 – 770 ) was a prominent Chinese poet of the Tang dynasty . Along with Li Bai ( Li Po ) , he is frequently called the greatest of the Chinese poets . His greatest ambition was to serve his country as a successful civil servant , but he proved unable to make the necessary accommodations . His life , like the whole country , was devastated by the An Lushan Rebellion of 755 , and his last 15 years were a time of almost constant unrest . 
 Although initially he was little @-@ known to other writers , his works came to be hugely influential in both Chinese and Japanese literary culture . Of his poetic writing , nearly fifteen hundred poems have been preserved over the ages . He has been called the " Poet @-@ Historian " and the " Poet @-@ Sage " by Chinese critics , while the range of his work has allowed him to be introduced to W

In [10]:
# Drop pages that are 50 characters in size or less
dropped_pages = [page for page in pages if len(page) <= 50]
pages = [page for page in pages if len(page) > 50]
dropped_pages

['___Text___']

In [11]:
subgraphs = []
for page in tqdm(pages):
    subgraph = prompts.graphPrompt(page, model="llama3.1:8b", silent=False, json=False)
    subgraphs.append(subgraph)

100%|███████████████████████████████████████████| 20/20 [01:11<00:00,  3.59s/it]


In [12]:
for subgraph in subgraphs:
    print(subgraph)

Here is the extracted ontology in JSON format:

[
   {
       "node_1": "Du Fu",
       "node_2": "Chinese poet",
       "edge": "Du Fu was a prominent Chinese poet of the Tang dynasty."
   },
   {
       "node_1": "Du Fu",
       "node_2": "Li Bai",
       "edge": "Along with Li Bai, he is frequently called the greatest of the Chinese poets."
   },
   {
       "node_1": "Du Fu",
       "node_2": "Chinese poet of the Tang dynasty",
       "edge": "He was a prominent Chinese poet of the Tang dynasty."
   },
   {
       "node_1": "Life of Du Fu",
       "node_2": "An Lushan Rebellion",
       "edge": "His life, like the whole country, was devastated by the An Lushan Rebellion of 755."
   },
   {
       "node_1": "Du Fu",
       "node_2": "Civil servant",
       "edge": "His greatest ambition was to serve his country as a successful civil servant."
   },
   {
       "node_1": "Works of Du Fu",
       "node_2": "Chinese literary culture",
       "edge": "Although initially he was little kn

In [13]:
def fix_prompt_output(text):
    """Fixes the prompt output by removing text that would produce a malfromed JSON."""
    # any line not starting with these characters is an LLM comment and not a JSON text line
    starting_characters = ('"', '{', '}', '[', ']')
    lines = text.splitlines()
    filtered_lines = [line for line in lines if any(line.lstrip().startswith(char) for char in starting_characters)]
    return "\n".join(filtered_lines)

In [14]:
fixed_subgraphs = []
for subgraph in subgraphs:
    fixed_subgraph = fix_prompt_output(subgraph)
    fixed_subgraphs.append(fixed_subgraph)
    print(fixed_subgraph)

[
   {
       "node_1": "Du Fu",
       "node_2": "Chinese poet",
       "edge": "Du Fu was a prominent Chinese poet of the Tang dynasty."
   },
   {
       "node_1": "Du Fu",
       "node_2": "Li Bai",
       "edge": "Along with Li Bai, he is frequently called the greatest of the Chinese poets."
   },
   {
       "node_1": "Du Fu",
       "node_2": "Chinese poet of the Tang dynasty",
       "edge": "He was a prominent Chinese poet of the Tang dynasty."
   },
   {
       "node_1": "Life of Du Fu",
       "node_2": "An Lushan Rebellion",
       "edge": "His life, like the whole country, was devastated by the An Lushan Rebellion of 755."
   },
   {
       "node_1": "Du Fu",
       "node_2": "Civil servant",
       "edge": "His greatest ambition was to serve his country as a successful civil servant."
   },
   {
       "node_1": "Works of Du Fu",
       "node_2": "Chinese literary culture",
       "edge": "Although initially he was little known to other writers, his works came to be hugel

In [15]:
parsed_subgraphs = []
for fixed_subgraph in fixed_subgraphs:
    try:
        parsed_subgraph = json.loads(fixed_subgraph)
    except json.decoder.JSONDecodeError as e:
        print(e)
        continue
    except Exception as e:
        print("Unexpected Exception:\n", e)
        continue
    parsed_subgraphs.append(parsed_subgraph)
    

In [16]:
parsed_graph = list(itertools.chain(*parsed_subgraphs))  # join the subgraphs to create a whole graph

In [17]:
len(parsed_graph)

171

In [18]:
parsed_graph

[{'node_1': 'Du Fu',
  'node_2': 'Chinese poet',
  'edge': 'Du Fu was a prominent Chinese poet of the Tang dynasty.'},
 {'node_1': 'Du Fu',
  'node_2': 'Li Bai',
  'edge': 'Along with Li Bai, he is frequently called the greatest of the Chinese poets.'},
 {'node_1': 'Du Fu',
  'node_2': 'Chinese poet of the Tang dynasty',
  'edge': 'He was a prominent Chinese poet of the Tang dynasty.'},
 {'node_1': 'Life of Du Fu',
  'node_2': 'An Lushan Rebellion',
  'edge': 'His life, like the whole country, was devastated by the An Lushan Rebellion of 755.'},
 {'node_1': 'Du Fu',
  'node_2': 'Civil servant',
  'edge': 'His greatest ambition was to serve his country as a successful civil servant.'},
 {'node_1': 'Works of Du Fu',
  'node_2': 'Chinese literary culture',
  'edge': 'Although initially he was little known to other writers, his works came to be hugely influential in both Chinese and Japanese literary culture.'},
 {'node_1': 'Du Fu',
  'node_2': 'Poet Historian',
  'edge': "He has been call

In [19]:
df = pd.DataFrame(parsed_graph)

In [20]:
df

,node_1,node_2,edge
0,Du Fu,Chinese poet,Du Fu was a prominent Chinese poet of the Tang...
1,Du Fu,Li Bai,"Along with Li Bai, he is frequently called the..."
2,Du Fu,Chinese poet of the Tang dynasty,He was a prominent Chinese poet of the Tang dy...
3,Life of Du Fu,An Lushan Rebellion,"His life, like the whole country, was devastat..."
4,Du Fu,Civil servant,His greatest ambition was to serve his country...
5,Works of Du Fu,Chinese literary culture,Although initially he was little known to othe...
6,Du Fu,Poet Historian,He has been called the 'Poet Historian' by Chi...
7,Du Fu,Chinese Virgil,He has been introduced to Western readers as '...
8,Traditional Chinese literary criticism,Life of the author,emphasized when interpreting a work
9,Art,Morality,close links posited by traditional Chinese tho...


In [21]:
df_no_dups = df.drop_duplicates()

In [22]:
df_no_dups

,node_1,node_2,edge
0,Du Fu,Chinese poet,Du Fu was a prominent Chinese poet of the Tang...
1,Du Fu,Li Bai,"Along with Li Bai, he is frequently called the..."
2,Du Fu,Chinese poet of the Tang dynasty,He was a prominent Chinese poet of the Tang dy...
3,Life of Du Fu,An Lushan Rebellion,"His life, like the whole country, was devastat..."
4,Du Fu,Civil servant,His greatest ambition was to serve his country...
5,Works of Du Fu,Chinese literary culture,Although initially he was little known to othe...
6,Du Fu,Poet Historian,He has been called the 'Poet Historian' by Chi...
7,Du Fu,Chinese Virgil,He has been introduced to Western readers as '...
8,Traditional Chinese literary criticism,Life of the author,emphasized when interpreting a work
9,Art,Morality,close links posited by traditional Chinese tho...


In [23]:
print("Number of duplicates: ", len(df) - len(df_no_dups))

Number of duplicates:  0


In [33]:
os.chdir("notebooks")

In [31]:
# Helper Function for visualizing the graph
def show_pyvis_graph(df, src_col, edge_col, tgt_col):
    G = nx.from_pandas_edgelist(df, source=src_col, edge_attr=edge_col, target=tgt_col)
    
    net = Network(
        notebook=True,
        bgcolor="#1a1a1a",
        cdn_resources="remote",
        height="500px",
        width="100%",
        select_menu=True,
        font_color="#cccccc",
        # filter_menu=True,
    )

    net.from_nx(G)

    # Assign networkx graph edges labels to pyvis edges labels
    for edge in net.edges:
        edge["label"] = edge[edge_col]
    
    net.repulsion(node_distance=150, spring_length=400)
    # net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
    # net.show_buttons(filter_=["physics"])
    
    show_pyvis_graph.counter += 1
    return net.show(f"gen-graph-v1-{show_pyvis_graph.counter}.html")
show_pyvis_graph.counter = 0

In [34]:
show_pyvis_graph(df, "node_1", "edge", "node_2")

In [ ]:
# Restore back to Project Folder
os.chdir("..")